In [1]:
import sys
sys.executable

'/Users/t/miniconda3/envs/deepchem/bin/python'

In [2]:
import deepchem


/Users/t/miniconda3/envs/deepchem/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
import deepchem as dc
import numpy as np

In [4]:
x = np.random.random((4, 5))
y = np.random.random((4, 1))

In [5]:
dataset = dc.data.NumpyDataset(x, y)

In [6]:
print(dataset.X)

[[0.67751269 0.95682018 0.38112273 0.37647984 0.98828253]
 [0.78893559 0.5067754  0.94876632 0.4214364  0.16172191]
 [0.68159588 0.14590784 0.88415715 0.75093345 0.27179819]
 [0.25499749 0.38167001 0.37702218 0.60005565 0.0392679 ]]


In [7]:
np.array_equal(x, dataset.X)

True

In [9]:
tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21(reload=False)

Loading raw samples now.
shard_size: 8192
About to start loading CSV from /var/folders/y6/s1r3q87933s6gjdrtmts1npc0000gq/T/tox21.csv.gz
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
TIMING: featurizing shard 0 took 14.537 s
TIMING: dataset construction took 14.980 s
Loading dataset from disk.
TIMING: dataset construction took 0.583 s
Loading dataset from disk.
TIMING: dataset construction took 0.720 s
Loading dataset from disk.
TIMING: dataset construction took 0.305 s
Loading dataset from disk.
TIMING: dataset construction took 0.441 s
Loading dataset from disk.


In [10]:
tox21_tasks

[u'NR-AR',
 u'NR-AR-LBD',
 u'NR-AhR',
 u'NR-Aromatase',
 u'NR-ER',
 u'NR-ER-LBD',
 u'NR-PPAR-gamma',
 u'SR-ARE',
 u'SR-ATAD5',
 u'SR-HSE',
 u'SR-MMP',
 u'SR-p53']

In [11]:
len(tox21_tasks)

12

In [12]:
tox21_datasets

(<deepchem.data.datasets.DiskDataset at 0x1a2c45af10>,
 <deepchem.data.datasets.DiskDataset at 0x1a2c4e6b10>)

In [13]:
train_dataset, valid_dataset, test_dataset = tox21_datasets

In [14]:
train_dataset.X.shape

(6264, 1024)

In [15]:
train_dataset.w.shape

(6264, 12)

In [16]:
transformers

In [17]:
model = dc.models.MultitaskClassifier(n_tasks=12,
                                          n_features=1024,
                                          layer_sizes=[1000])

In [18]:
model.fit(train_dataset, nb_epoch=10)

857.2092112707713

In [20]:
?model.fit

In [21]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

In [22]:
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)

computed_metrics: [0.9894300497269047, 0.9959969091201029, 0.962168988387148, 0.9808138857089346, 0.9049994274336839, 0.9834026252762793, 0.99129239723575, 0.9107864058982535, 0.9856337388556753, 0.9696577207987092, 0.9463814211735747, 0.9751423616937964]
computed_metrics: [0.8001475742482937, 0.8554487719889844, 0.901396737392046, 0.8008687460749424, 0.7140896315823875, 0.7907240573907242, 0.7200430321085733, 0.7230549044356294, 0.8492629576795054, 0.7264337508239948, 0.8696998669770535, 0.7859848484848485]


In [23]:
print(train_scores)

{'mean-roc_auc_score': 0.9663088276090677}


In [24]:
print(test_scores)

{'mean-roc_auc_score': 0.7947629065989151}


In [25]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [26]:
import tensorflow as tf
import deepchem.models.tensorgraph.layers as layers

In [27]:
train_dataset = dc.data.NumpyDataset(mnist.train.images, mnist.train.labels)
test_dataset = dc.data.NumpyDataset(mnist.test.images, mnist.test.labels)

In [28]:
model = dc.models.TensorGraph(model_dir='mnist')


In [29]:
feature = layers.Feature(shape=(None, 784))

In [30]:
label = layers.Label(shape=(None, 10))

In [31]:
make_image = layers.Reshape(shape=(None, 28, 28), in_layers=feature)

In [32]:
conv2d_1 = layers.Conv2D(num_outputs=32, activation_fn=tf.nn.relu, in_layers=make_image)
conv2d_2 = layers.Conv2D(num_outputs=64, activation_fn=tf.nn.relu, in_layers=conv2d_1)

In [33]:
flatten = layers.Flatten(in_layers=conv2d_2)
dense1 = layers.Dense(out_channels=1024, activation_fn=tf.nn.relu, in_layers=flatten)
dense2 = layers.Dense(out_channels=10, activation_fn=None, in_layers=dense1)

In [34]:

smce = layers.SoftMaxCrossEntropy(in_layers=[label, dense2])
loss = layers.ReduceMean(in_layers=smce)
model.set_loss(loss)

In [35]:
output = layers.SoftMax(in_layers=dense2)
model.add_output(output)

In [ ]:
model.fit(train_dataset, nb_epoch=10)

In [ ]:
metric = dc.metrics.Metric(dc.metrics.accuracy_score)

In [ ]:
train_scores = model.evaluate(train_dataset, [metric])
test_scores = model.evaluate(test_dataset, [metric])